In [1]:
import xarray as xr
import pathlib as pl
import pandas as pd
import pywatershed
import os
import dask
import numpy as np

In [2]:
all_models = ['01473000','05431486','09112500','14015000']# Create a list of all cutouts

In [3]:
rootdir = pl.Path('../NHM_extractions/20230110_pois_haj/')# Path to location of cutouts

In [4]:
#var_output_files = ['hru_actet.nc', 'recharge.nc', 'soil_rechr.nc', 'snowcov_area.nc', 'seg_outflow.nc',]#output files of interest

### Working currently from a single cutout directory

In [5]:
cm = all_models[0] # sets cutout from list
outvardir = rootdir/ cm / 'output'# stes path to location of NHM output folder where output files are.

In [6]:
# set the file name for the postprocessed model output file that PEST will read
of_name = 'modelobs.dat'# name of file

In [7]:
########### Useful code
#indat
#df.index.dtypes
#xr.__version__

In [8]:
#all_nc_file = sorted([i for i in (rootdir/ cm / 'output').glob('model_custom_output.nc')])#Read in the files to check

In [9]:
# make a file to hold the consolidated results
ofp = open(rootdir / cm / 'modelobs.dat', 'w') # the 'w' will delete any existing file here and recreate; 'a' appends

In [10]:
modelobsdat  = xr.open_dataset(outvardir / 'model_custom_output.nc')

In [11]:
#modelobsdat

### Slice output to calibration periods for each variable
#### These are as follows from table 2 (Hay and others, 2023):


In [12]:
aet_start = '2000-01-01'
aet_end = '2010-12-31'
recharge_start = '2000-01-01'
recharge_end = '2009-12-31'
runoff_start = '1982-01-01'
runoff_end = '2010-12-31'
soil_rechr_start = '1982-01-01'
soil_rechr_end = '2010-12-31'
sca_start = '2000-01-01'
sca_end = '2010-12-31'
seg_outflow_start = '2000-01-01'
seg_outflow_end = '2010-12-31'

### Actual ET
#### Get and check the daily data

In [13]:
#actet_daily = (xr.open_dataset(outvardir / 'hru_actet.nc')['hru_actet']).sel(time=slice(aet_start, aet_end))
actet_daily = modelobsdat.hru_actet.sel(time=slice(aet_start, aet_end))

In [14]:
#actet_daily

#### Post-process daily output to match observation targets of "monthly" and "mean monthly"

In [15]:
#Creates a dataframe time series of monthly values (average daily rate for the month)
actet_monthly = actet_daily.resample(time = 'm').mean()

In [16]:
#actet_monthly

In [17]:
# Creates a dataframe time series of mean monthly (mean of all jan, feb, mar....)
actet_mean_monthly = actet_monthly.groupby('time.month').mean()

In [18]:
#actet_mean_monthly

In [19]:
### Used for plotting/checking
#df = actet_monthly.to_dataframe()# makes a dataframe for plotting and writing out
#df_r = np.ravel(actet_monthly_df, order = 'C')# flattens the 2D array to a 1D array--just playing
#df[8465:8473]
#df.xs(5625, level=1).plot()

#### Now write values to the template file

In [20]:
inds = [f'{i.year}_{i.month}:{j}' for i in actet_monthly.indexes['time'] for j in actet_monthly['nhm_id'].values]# set up the indices in sequence
varvals = np.ravel(actet_monthly, order = 'C')# flattens the 2D array to a 1D array--just playing 

with open(rootdir / cm / of_name, encoding="utf-8", mode='w') as ofp:
    ofp.write('obsname    obsval\n') # writing a header for the file
    [ofp.write(f'actet_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [21]:
actet_monthly.sel(time='2000-01-31').values # look at a slice of the netcdf and compare to pest write

array([0.01872298, 0.01335401, 0.02401422, 0.02302729, 0.02104963,
       0.02078296, 0.0102255 , 0.00803491, 0.01745123, 0.01889673,
       0.02125189, 0.01306235, 0.01861553, 0.01775   , 0.01272994,
       0.01571974, 0.01441734])

In [22]:
[(i,j) for i,j in zip(inds,varvals)] # playing around and learning adv lists making

[('2000_1:5621', 0.018722979149330198),
 ('2000_1:5625', 0.013354014934519418),
 ('2000_1:5628', 0.024014221169636182),
 ('2000_1:5635', 0.02302728590993128),
 ('2000_1:5637', 0.0210496318661405),
 ('2000_1:5643', 0.020782960068698165),
 ('2000_1:5678', 0.010225496307092958),
 ('2000_1:5679', 0.008034910561232384),
 ('2000_1:5686', 0.017451230870733893),
 ('2000_1:5690', 0.018896731326354506),
 ('2000_1:5693', 0.021251885933652865),
 ('2000_1:5697', 0.01306235112462228),
 ('2000_1:5703', 0.01861552880539323),
 ('2000_1:5728', 0.017749996019908548),
 ('2000_1:7128', 0.012729942847455966),
 ('2000_1:7156', 0.015719744154768397),
 ('2000_1:7157', 0.014417340475473217),
 ('2000_2:5621', 0.044012876260611275),
 ('2000_2:5625', 0.03588178741887825),
 ('2000_2:5628', 0.05329765743755632),
 ('2000_2:5635', 0.04928992969088599),
 ('2000_2:5637', 0.04941284607596243),
 ('2000_2:5643', 0.04697932902909509),
 ('2000_2:5678', 0.029800105728549225),
 ('2000_2:5679', 0.025387249882439445),
 ('2000_2:

In [23]:
inds = [f'{i}:{j}' for i in actet_mean_monthly.indexes['month'] for j in actet_mean_monthly['nhm_id'].values]
varvals =  np.ravel(actet_mean_monthly, order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'actet_mean_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [24]:
#actet_mean_monthly

In [25]:
#actet_mean_monthly.sel(month=1).values # look at a slice of the netcdf and compare to pest write file

### Post Process recharge for calibration use
#### Get daily output file from NHM for recharge

In [26]:
#recharge_daily = (xr.open_dataset(outvardir / 'recharge.nc')['recharge']).sel(time=slice(recharge_start, recharge_end))
recharge_daily = modelobsdat.recharge.sel(time=slice(recharge_start, recharge_end))

#### Post-process daily output to match observation target of "annual recharge" as an average daily rate for the year

In [27]:
recharge_annual = recharge_daily.resample(time = 'Y').mean()
recharge_annual_norm = (recharge_annual - recharge_annual.min())/(recharge_annual.max()-recharge_annual.min())

#### Write values to template file

In [28]:
inds = [f'{i.year}:{j}' for i in recharge_annual_norm.indexes['time'] for j in recharge_annual_norm['nhm_id'].values]
varvals =  np.ravel(recharge_annual_norm, order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'recharge_ann:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [29]:
recharge_annual_norm

<xarray.DataArray 'recharge' (time: 10, nhru: 17)>
array([[0.34043334, 0.34050935, 0.0366373 , 0.41873038, 0.10847274,
        0.31339277, 0.15571652, 0.50865955, 0.25062198, 0.26792908,
        0.03623859, 0.19469808, 0.31600034, 0.27905126, 0.69723237,
        0.14610309, 0.33220391],
       [0.23275807, 0.29909385, 0.04609285, 0.35312459, 0.08953394,
        0.25687275, 0.07926453, 0.35282931, 0.13909208, 0.19625334,
        0.02728602, 0.12722969, 0.22711517, 0.22492658, 0.6463799 ,
        0.08331952, 0.27560446],
       [0.37926354, 0.40756816, 0.08881   , 0.44698063, 0.18417041,
        0.36316589, 0.17546604, 0.49379695, 0.2655714 , 0.26833276,
        0.04887508, 0.17310897, 0.32991925, 0.25546886, 0.69295691,
        0.21427385, 0.35390583],
       [0.76548016, 0.76467277, 0.25950215, 0.79938717, 0.46183404,
        0.6770035 , 0.33633185, 0.84469192, 0.55328715, 0.56163579,
        0.09761168, 0.49408528, 0.69421623, 0.43686606, 1.        ,
        0.54897887, 0.71477245],
       [0.45400363, 0.50623944, 0.12220297, 0.57460179, 0.30000211,
        0.46214742, 0.21471877, 0.67919772, 0.40931406, 0.37130375,
        0.03664696, 0.34989583, 0.45312052, 0.34931542, 0.85883001,
        0.39067275, 0.55531123],
       [0.41965223, 0.43982379, 0.08893388, 0.50919642, 0.24604695,
        0.40484397, 0.1392652 , 0.48745978, 0.30667116, 0.2997512 ,
        0.02193447, 0.2663045 , 0.39575559, 0.26380476, 0.70355774,
        0.30266438, 0.44099768],
       [0.60501734, 0.55834861, 0.15171715, 0.61615883, 0.28697569,
        0.5002892 , 0.24196894, 0.64141032, 0.40285828, 0.32096311,
        0.0780489 , 0.37442488, 0.58163697, 0.40690546, 0.95858889,
        0.39229026, 0.52334376],
       [0.33760501, 0.39840462, 0.06872002, 0.43238169, 0.20580267,
        0.35545799, 0.12370237, 0.43264368, 0.26144572, 0.2780321 ,
        0.        , 0.22122483, 0.35817945, 0.21993081, 0.70984519,
        0.2221802 , 0.3529018 ],
       [0.45883019, 0.45776329, 0.10520078, 0.54773659, 0.23284084,
        0.43889584, 0.19953316, 0.58442676, 0.33609462, 0.35379489,
        0.05859347, 0.29331798, 0.47221612, 0.34693245, 0.87174416,
        0.3228458 , 0.48832723],
       [0.56481972, 0.55740552, 0.14043508, 0.58464478, 0.25541437,
        0.47047596, 0.29162798, 0.6971613 , 0.39775585, 0.33674055,
        0.09628232, 0.33189813, 0.49549617, 0.35369603, 0.89087267,
        0.40957986, 0.5597406 ]])
Coordinates:
    nhm_id   (nhru) int64 5621 5625 5628 5635 5637 ... 5703 5728 7128 7156 7157
  * time     (time) datetime64[ns] 2000-12-31 2001-12-31 ... 2009-12-31
Dimensions without coordinates: nhru

In [30]:
#recharge_annual_norm.sel().values # look at a slice of the netcdf and compare to pest write file

### Post Process "soil_rechr" to compare to target
#### Get daily output file from NHM for soil recharge and normalize 0-1

In [31]:
#soil_rechr_daily = (xr.open_dataset(outvardir / 'soil_rechr.nc')['soil_rechr']).sel(time=slice(soil_rechr_start, soil_rechr_end))
soil_rechr_daily = modelobsdat.soil_rechr.sel(time=slice(soil_rechr_start, soil_rechr_end))

In [32]:
#Creates a dataframe time series of monthly values (average daily rate for each month)
soil_rechr_monthly = soil_rechr_daily.resample(time = 'm').mean()
soil_rechr_monthly_norm = (soil_rechr_monthly - soil_rechr_monthly.min())/(soil_rechr_monthly.max()-soil_rechr_monthly.min())

#Creates a dataframe time series of annual values (average daily value for each year)
soil_rechr_annual = soil_rechr_daily.resample(time = 'Y').mean()
soil_rechr_annual_norm = (soil_rechr_annual - soil_rechr_annual.min())/(soil_rechr_annual.max()-soil_rechr_annual.min())

In [33]:
inds = [f'{i.year}_{i.month}:{j}' for i in soil_rechr_monthly_norm.indexes['time'] for j in soil_rechr_monthly_norm['nhm_id'].values]
varvals = np.ravel(soil_rechr_monthly_norm, order = 'C')# flattens the 2D array to a 1D array

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'soil_moist_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [34]:
inds = [f'{i.year}:{j}' for i in soil_rechr_annual_norm.indexes['time'] for j in soil_rechr_annual_norm['nhm_id'].values]
varvals =  np.ravel(soil_rechr_annual_norm, order = 'C')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'soil_moist_ann:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [35]:
soil_rechr_annual_norm

<xarray.DataArray 'soil_rechr' (time: 29, nhru: 17)>
array([[0.43200886, 0.81186523, 0.63830307, 0.49692973, 0.65216655,
        0.73551993, 0.71648003, 0.71721478, 0.5441952 , 0.57805178,
        0.39776468, 0.84839449, 0.54314662, 0.29234995, 0.01588402,
        0.43470906, 0.78613155],
       [0.40235666, 0.7457123 , 0.59995377, 0.46885214, 0.62543875,
        0.69379692, 0.69171296, 0.68243089, 0.52531292, 0.57468217,
        0.38849929, 0.80825577, 0.52192208, 0.28649925, 0.01574534,
        0.41743709, 0.76614721],
       [0.46016596, 0.84263128, 0.67109793, 0.53083578, 0.69299384,
        0.77433378, 0.78751287, 0.77547912, 0.59371575, 0.63911716,
        0.44884256, 0.88196854, 0.58079505, 0.33040988, 0.02829973,
        0.48121323, 0.83461366],
       [0.38864206, 0.78953261, 0.5825259 , 0.46430413, 0.59262002,
        0.70532756, 0.72181083, 0.75205189, 0.51837471, 0.54481855,
        0.37000133, 0.83843564, 0.4858641 , 0.26867079, 0.00459736,
        0.39646268, 0.77938416],
       [0.39679535, 0.77305392, 0.61127673, 0.46604265, 0.62473116,
        0.69854247, 0.7160817 , 0.72595056, 0.52392584, 0.55408145,
        0.376037  , 0.82407537, 0.50763962, 0.27578674, 0.01197198,
        0.41176779, 0.76718558],
...
       [0.43583481, 0.85844113, 0.66055497, 0.52641458, 0.69820386,
        0.79783498, 0.81168756, 0.79943758, 0.589941  , 0.67666653,
        0.44163527, 0.93006651, 0.5775974 , 0.31871274, 0.01821315,
        0.46450446, 0.862226  ],
       [0.37459897, 0.73320211, 0.56509063, 0.45670702, 0.59569851,
        0.67987885, 0.72046264, 0.72020066, 0.52169482, 0.57568847,
        0.38901342, 0.82408012, 0.510413  , 0.2821548 , 0.00930748,
        0.41390954, 0.77484195],
       [0.42832546, 0.83434326, 0.6630698 , 0.52015455, 0.68764382,
        0.7759123 , 0.79314403, 0.78100251, 0.58399013, 0.65416033,
        0.43381877, 0.89170782, 0.54019728, 0.31383106, 0.01029142,
        0.45354114, 0.8405129 ],
       [0.48806994, 0.93628285, 0.7219204 , 0.58962652, 0.76873108,
        0.8692788 , 0.88664185, 0.85584225, 0.65393315, 0.74776447,
        0.50449756, 1.        , 0.63629092, 0.36967681, 0.03650786,
        0.51945784, 0.95078307],
       [0.39793981, 0.78148137, 0.59568306, 0.48169277, 0.62675789,
        0.73012402, 0.71766488, 0.71401054, 0.53781232, 0.59072571,
        0.38839224, 0.83236689, 0.50601907, 0.27111802, 0.00233642,
        0.42265227, 0.78273405]])
Coordinates:
    nhm_id   (nhru) int64 5621 5625 5628 5635 5637 ... 5703 5728 7128 7156 7157
  * time     (time) datetime64[ns] 1982-12-31 1983-12-31 ... 2010-12-31
Dimensions without coordinates: nhru

In [36]:
#soil_rechr_monthly_norm.sel(time='1982-01-31').values # look at a slice of the netcdf and compare to pest write file
#soil_rechr_annual_norm.sel(time='1982-12-31').values # look at a slice of the netcdf and compare to pest write file

In [37]:
#actet_mean_monthly.sel(month=1).values # look at a slice of the netcdf and compare to pest write file

### Post Process "hru_outflow" to compare to target
#### Get and check the daily data

In [38]:
# These units are in cubic feet (implied per day)
hru_streamflow_out_daily = modelobsdat.hru_streamflow_out.sel(time=slice(runoff_start, runoff_end))

In [39]:
hru_streamflow_out_monthly = hru_streamflow_out_daily.resample(time = 'm').mean()

In [40]:
#This converts the average daily rate to a rate in cubic feet per second to compare to observation
hru_streamflow_out_rate = (hru_streamflow_out_monthly)/(24*60*60)

In [41]:
inds = [f'{i.year}_{i.month}:{j}' for i in hru_streamflow_out_rate.indexes['time'] for j in hru_streamflow_out_rate['nhm_id'].values]
varvals = np.ravel(hru_streamflow_out_rate, order = 'C')# flattens the 2D array to a 1D array

with open(rootdir / cm / of_name, encoding="utf-8",mode='a') as ofp:
    [ofp.write(f'runoff_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [42]:
#hru_streamflow_out_rate

In [43]:
#hru_streamflow_out_rate.sel(time='1982-01-31T00:00:00.000000000').values # look at a slice of the netcdf and compare to pest write file

### Post Process "snowcov_area" to compare to target
#### Get and check the daily data

In [44]:
#snowcov_area_daily = (xr.open_dataset(outvardir / 'snowcov_area.nc')['snowcov_area']).sel(time=slice(sca_start, sca_end))
snowcov_area_daily = modelobsdat.snowcov_area.sel(time=slice(sca_start, sca_end))
remove_ja = True #This is used the filter for removing July and August from the dataset

In [45]:
#Applying filter to remove months, July and August, from the dataset so same size as obs data.
if remove_ja:
    snowcov_area_daily_restr = snowcov_area_daily.sel(time=snowcov_area_daily.time.dt.month.isin([1, 2, 3, 4, 5, 6, 9, 10, 11, 12]))
else:
    snowcov_area_daily_restr = snowcov_area_daily
snowcov_area_daily.close()     

In [46]:
inds = [f'{i.year}_{i.month}_{i.day}:{j}' for i in snowcov_area_daily_restr.indexes['time'] for j in snowcov_area_daily_restr['nhm_id'].values]
varvals = np.ravel(snowcov_area_daily_restr, order = 'C')# flattens the 2D array to a 1D array

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'sca_daily:{i}          {j}\n') for i,j in zip(inds,varvals)]

### Get the daily streamflow values from segments associated with the gage pois

In [47]:
# Get seg_outflow data
#seg_outflow_daily = (xr.open_dataset(outvardir / 'seg_outflow.nc')['seg_outflow']).sel(time=slice(seg_outflow_start, seg_outflow_end),nhm_seg= poi_gage_nhm_seg)
seg_outflow_daily = modelobsdat.seg_outflow.sel(time=slice(seg_outflow_start, seg_outflow_end))

In [48]:
seg_outflow_daily

<xarray.DataArray 'seg_outflow' (time: 4018, npoi_gages: 5)>
[20090 values with dtype=float64]
Coordinates:
  * time       (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2010-12-31
    poi_gages  (npoi_gages) object ...
Dimensions without coordinates: npoi_gages
Attributes:
    desc:     Streamflow leaving a segment
    units:    cfs

In [49]:
inds = [f'{i.year}_{i.month}_{i.day}:{j}' for j in seg_outflow_daily['poi_gages'].values for i in seg_outflow_daily.indexes['time']]
varvals = np.ravel(seg_outflow_daily, order = 'F')# flattens the 2D array to a 1D array

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'streamflow_daily:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [50]:
seg_outflow_daily

<xarray.DataArray 'seg_outflow' (time: 4018, npoi_gages: 5)>
array([[ 26.311771,  13.218765,  43.104563,   6.388665, 179.894947],
       [ 26.039267,  12.890276,  40.383779,   6.229941, 172.916772],
       [ 24.979324,  12.637366,  39.135089,   6.093091, 167.321246],
       ...,
       [ 23.438487,  13.425328,  37.085038,   5.682206, 156.975848],
       [ 22.965587,  13.148623,  36.278181,   5.57878 , 153.300779],
       [ 22.539987,  12.902473,  35.558863,   5.48644 , 149.990253]])
Coordinates:
  * time       (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2010-12-31
    poi_gages  (npoi_gages) object '01472198' '01472199' ... '01473000'
Dimensions without coordinates: npoi_gages
Attributes:
    desc:     Streamflow leaving a segment
    units:    cfs

In [51]:
#seg_outflow_daily.sel(npoi_gages='01472198').values # look at a slice of the netcdf and compare to pest write file

#### Post-process daily output to match observation targets of "monthly" and "mean monthly"

In [52]:
#Creates a dataframe time series of monthly values (average daily rate for the month)
seg_outflow_monthly = seg_outflow_daily.resample(time = 'm').mean()

In [53]:
# Creates a dataframe time series of mean monthly (mean of all jan, feb, mar....)
seg_outflow_mean_monthly = seg_outflow_monthly.groupby('time.month').mean()

In [54]:
#Now write to the pest obs file
inds = [f'{i.year}_{i.month}:{j}' for j in seg_outflow_monthly['poi_gages'].values for i in seg_outflow_monthly.indexes['time'] ]# set up the indices in sequence
varvals = np.ravel(seg_outflow_monthly, order = 'F')# flattens the 2D array to a 1D array--just playing 

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'streamflow_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]

In [55]:
inds = [f'{i}:{j}' for j in seg_outflow_mean_monthly['poi_gages'].values for i in seg_outflow_mean_monthly.indexes['month'] ]
varvals =  np.ravel(seg_outflow_mean_monthly, order = 'F')# flattens the 2D array to a 1D array 

with open(rootdir / cm / of_name, encoding="utf-8", mode='a') as ofp:
    [ofp.write(f'streamflow_mean_mon:{i}          {j}\n') for i,j in zip(inds,varvals)]